# CarND Behavioral Cloning Project

This project is about training a neural network to drive a car on a simulator using data recorded from a humman driver.

This notebook will be used to create the model to be used in driving the car on the simulator.

The inputs come in three images right, central and left cameras. I decided to use only the center one, because the get good valid values for the left and right images was a bit dificult.

The first thing to do is to clean, then oganize the dataset, and save it to a csv file. for posterior use.

The file driving_log.csv contains steering angles and the left, right and center images associated to it.

In [ ]:
import csv
import numpy as np
import cv2
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import train_test_split
import os.path
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(42)

### Dataset preparation

I first get all records regarding the center image and shufle them.

Then I split the data into train 80% and validation 20%

Next I save the train and validation array to csv files

In [ ]:


if not(os.path.exists('train.csv') and os.path.exists('validation.csv')):

    path_to_replace = "C:\\Users\\eduardo\\Documents\\SelfDrivingCar\\beta-simulator-windows\\beta_simulator_windows\\data"

    def ReplaceWrongPath(value):
        return value.replace(path_to_replace, "").replace("\\", "/").replace(" ", "")

    X_train = []
    y_train = []
    X_train_left = []
    y_train_left = []
    X_train_right = []
    y_train_right = []

    with open('./data/driving_log.csv', 'r') as csv_file_in:

        csv_reader = csv.DictReader(csv_file_in)

        for row in csv_reader:
            steering = float(row['steering'])

            #center image
            path = './data/' + ReplaceWrongPath(row['center'].strip())        
            X_train.append(path)
            y_train.append(steering)

    X_train, y_train = shuffle(X_train, y_train)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    [X_train.append(item) for item in X_train_left]
    [X_train.append(item) for item in X_train_right]
    [y_train.append(item) for item in y_train_left]
    [y_train.append(item) for item in y_train_right]
    
    X_train, y_train = shuffle(X_train, y_train)

    with open('train.csv', 'w') as csv_file_train:

        fieldnames = ['path','steering']
        writer = csv.DictWriter(csv_file_train, fieldnames=fieldnames)
        writer.writeheader()

        for i in range(len(X_train)):
            path, steering = X_train[i], y_train[i]
            writer.writerow({'path': path, 'steering': steering})

    with open('validation.csv', 'w') as csv_file_train:

        fieldnames = ['path','steering']
        writer = csv.DictWriter(csv_file_train, fieldnames=fieldnames)
        writer.writeheader()

        for i in range(len(X_validation)):
            path, steering = X_validation[i], y_validation[i]
            writer.writerow({'path': path, 'steering': steering})

    print("processing done")
else:
    print("files exist")

### Load data 

In this step the csv files into memory, they are already pre shufled and split

In [ ]:
import sys
    
    
break_classes = 21
X_train_left = None
y_train_left = None
X_train_right = None
y_train_right = None

X_train= []
y_train = []

with open('train.csv', 'r') as csv_file_train:
    
    csv_reader = csv.DictReader(csv_file_train)

    for row in csv_reader:
        X_train.append(row['path'])
        y_train.append(float(row['steering']))
        

X_validation = []
y_validation = []

with open('validation.csv', 'r') as csv_file_val:
    
    csv_reader = csv.DictReader(csv_file_val)

    for row in csv_reader:
        X_validation.append(row['path'])
        y_validation.append(float(row['steering']))
        
print('Loading done')
        

### Explore data

Here I do histogram on the training target, this is also how I try to check if the recorded data is balanced enough, to produce a balanced solution.

In [ ]:
plt.hist(y_train, bins=70)
plt.title("Training data")
plt.xlabel("Angles")
plt.ylabel("# of examples")
plt.show()

# Model

For the model I decided to experiment with the VGG16 pre-trained model.

By using a pre-trained VGG16 I could get a working model in just one single epoch.

I removed the top fully connected layer, and placed a 4 layers fully connect NN.

But I also experimented with other models, that also performed well but not as well as the VGG.

Relu layers were used to introduce non linearity.

## Chalenges

### Data Colection

For me the bigest chalenge was collecting the right data to train the model.

I had to redo the dataset a few times either because sometimes I had too much recovery examples or too little recovery examples. But in the begining this was a major set back because I thought that the model was the problem, when in reality the problem was the incredibly imbalanced data that I had.

My Dataset is composed of:
- Two center driving laps on the correct direction.
- Two center driving laps on the wrong direction.
- One lap revering from the right on the correct direction
- One lap revering from the right on the wrong direction
- One lap revering from the left on the correct direction
- One lap revering from the left on the wrong direction
- Two other trys on the curves

### Memory limit

I also faced a few issues with memory.

First it's not possible to load the entire dataset into memmory, so a fit generator was used to create the train and validation set, I also flipped the image horizontaly doubling the dataset size, also in the hope of introducing some generalization with it.

Second I had to use smaler batch sizes, because tha GPU that I was using couldn't handle much data.

## Croping

The original image has a shape of (160, 320, 3) than I've croped 60px from the top and 20px from the bottom.

I've also used the Lambda layer after to normalize the image


![alt text](./model.png "Model Visualization")


# Training

I chose to use a RMSprop optimizer.

And ModelCheckpoint to save the best evaluated model.

Tests were done straight into the simulator.



In [1]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Lambda, Dense, Activation, Flatten, Input, Dropout, Convolution2D, MaxPooling2D, Cropping2D
from keras.layers.normalization import BatchNormalization

def get_model():
    input_tensor = Input(shape=(160, 320, 3))
    croped_input_img = Cropping2D(cropping=((60, 20), (0, 0)))(input_tensor)
    croped_input_img = Lambda(lambda x: (x / 255.0) - 0.5)(croped_input_img)
    base_model = VGG16(input_tensor=croped_input_img, weights='imagenet', include_top=False)

    for layer in base_model.layers:
        layer.trainable = False

    top_model = base_model.output
    top_model = Flatten()(top_model)
    top_model = Dropout(0.25)(top_model)
    
    top_model = Dense(2024)(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Activation('relu')(top_model)
    top_model = Dropout(0.25)(top_model)

    top_model = Dense(500)(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Activation('relu')(top_model)
    top_model = Dropout(0.25)(top_model)

    top_model = Dense(200)(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Activation('relu')(top_model)
    top_model = Dropout(0.25)(top_model)

    top_model = Dense(50)(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Activation('relu')(top_model)
    top_model = Dropout(0.25)(top_model)

    predictions = Dense(1)(top_model)

    model = Model(input=input_tensor, output=predictions)
    return model

Using TensorFlow backend.


In [2]:
import pydot

print(pydot.find_graphviz())

from keras.utils.visualize_util import plot

model = get_model()

plot(model, to_file='model.png')

{'sfdp': 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\sfdp.exe', 'neato': 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\neato.exe', 'fdp': 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\fdp.exe', 'twopi': 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\twopi.exe', 'dot': 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\dot.exe', 'circo': 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\circo.exe'}


In [ ]:
def process_line(row):
    #b,g,r = cv2.split(cv2.imread(row['path']))
    #img = np.array(cv2.merge([r,g,b]))
    img = cv2.imread(row['path'])
    steering = float(row['steering'])
    return [img, steering]

def generate_arrays_from_file(path, batch_size = 20, flip=True):
    while 1:
        global X_train
        global y_train
        X_train, y_train = shuffle(X_train, y_train)
        Xs = []
        ys = []        
        for i in range(len(X_train)):
            if (len(Xs) == batch_size):
                yield (np.array(Xs), np.array(ys))
                Xs = []
                ys = []
                
            x, y = process_line({'path':X_train[i], 'steering':y_train[i]})
            Xs.append(x)
            ys.append(y)
            
            if flip:
                if (len(Xs) == batch_size):
                    yield (np.array(Xs), np.array(ys))
                    Xs = []
                    ys = []
                    
                x_flipped = np.fliplr(x)
                y_filpped = -y

                Xs.append(x_flipped)
                ys.append(y_filpped)

        yield (np.array(Xs), np.array(ys))
            

In [ ]:
train_rows = len(X_train)
validation_rows = len(X_validation)
    
print('train records: ', train_rows)
print('validation records: ', validation_rows)

In [ ]:
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint

model = get_model()
optimizer = RMSprop(lr=0.0001)
model.compile(loss='mean_absolute_error', optimizer=optimizer)
checkpoint = ModelCheckpoint('model1.h5', monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='auto')
history = model.fit_generator(
    generate_arrays_from_file('train.csv',batch_size=100), 
    samples_per_epoch=train_rows*2, 
    nb_epoch=30, 
    validation_data=generate_arrays_from_file('validation.csv', batch_size=10, flip=True),
    nb_val_samples=validation_rows*2, 
    callbacks=[checkpoint])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()